# Microsoft model

In [ ]:
#import the libraries
from transformers import AutoImageProcessor, TableTransformerModel, TableTransformerConfig
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
model_name = "microsoft/table-transformer-structure-recognition" #model used
# Load the image processor and model
image_processor = AutoImageProcessor.from_pretrained(model_name)
model_config = TableTransformerConfig.from_pretrained(model_name)
model = TableTransformerModel.from_pretrained(model_name, config=model_config)

In [ ]:
image_path = r"C:\Users\amana\OneDrive\Pictures\Screenshots\Screenshot 2024-02-06 213723.png"
image = Image.open(image_path).convert('RGB')
inputs = image_processor(images=image, return_tensors='pt')
outputs = model(**inputs)
# Convert the output tensors to an image
processed_image = inputs["pixel_values"].squeeze().permute(1, 2, 0).numpy().astype("uint8")
plt.imshow(processed_image)
plt.axis('off')
plt.show()

# YOLOV8X

In [ ]:
import ultralytics as ul
model=ul.YOLO('yolov8x.pt')
im_path=r"C:\Users\amana\OneDrive\Pictures\Screenshots\Screenshot 2024-02-07 082525.png"
res=model(im_path)

In [ ]:
res=res[0]
boxes=res.boxes.xyxy
score=res.boxes.conf
render=ul.render_result(model=model,image=im_path,result=res)
render.show()
### not good at detecting the table

# keremberke(this model actually works)

In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install --upgrade tensorboard

In [ ]:
pip install ultralyticsplus==0.0.23 ultralytics==8.0.21

In [ ]:
pip install numpy==1.24.4

In [ ]:
from ultralyticsplus import YOLO, render_result
model= YOLO('keremberke/yolov8m-table-extraction')

In [ ]:
#testing it out for a single pic
image=r"C:\Users\amana\OneDrive\Desktop\pdf images\ABR_011_Suzhou\21675.pdf\1.jpg"
res=model.predict(image)

In [ ]:
render=render_result(model=model,image=image,result=res[0])
render.show()  # display the image

# Using keremberke for all the pdfs in the folder and finding accuracy

In [ ]:
import os
from pdf2image import convert_from_path
main_folder=r"C:\Users\amana\OneDrive\Desktop\ASIA\ASIA"
image_folder=r"C:\Users\amana\OneDrive\Desktop\pdf images"  #making a seperate folder to store images of the pdf files.

In [ ]:
# for x in os.listdir(main_folder):
#     fol=os.path.join(image_folder,x)
#     os.makedirs(fol,exist_ok=True)
#     x_path=os.path.join(main_folder,x)
#     for y in os.listdir(x_path):
#         if y.endswith(".pdf"):
#             pdf_image_folder=os.path.join(fol,y+"images")
#             images = convert_from_path(y, poppler_path=r"C:\Users\amana\Downloads\Release-23.11.0-0\poppler-23.11.0\Library\bin")
#             for index, im in enumerate(images):
#                 image_path = os.path.join(pdf_image_folder, f'{os.path.splitext(os.path.basename(pdf_path))[0]}-{index}.png')
#                 image.save(image_path)           

In [ ]:
# for x in os.listdir(main_folder):
#     fol = os.path.join(image_folder, x)
#     os.makedirs(fol, exist_ok=True)
#     x_path = os.path.join(main_folder, x)
#     for y in os.listdir(x_path):
#         if y.endswith(".pdf"):
#             pdf_path = os.path.join(x_path, y)  # full path of the pdf file
#             pdf_image_folder = os.path.join(fol,y)
#             os.makedirs(pdf_image_folder,exist_ok=True)
#             images = convert_from_path(pdf_path, poppler_path=r"C:\Users\amana\Downloads\Release-23.11.0-0\poppler-23.11.0\Library\bin")
#             for index, image in enumerate(images):
#                 image.save(pdf_image_folder)

In [ ]:
def process_pdf_files(folder_path, output_folder):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".pdf"):
                pdf_path = os.path.join(root, file)  # pdf file path
                pdf_image_folder = os.path.join(output_folder, os.path.relpath(root, folder_path), file) # path for pdf images
                try:
                    images = convert_from_path(pdf_path, poppler_path=r"C:\Users\amana\Downloads\Release-23.11.0-0\poppler-23.11.0\Library\bin")
                    for index, image in enumerate(images):
                        result=model.predict(image)
                        render=render_result(model=model,image=image,result=result[0])
                        if len(result[0].boxes.xyxy) > 0:
                            render.save(pdf_image_folder)
                except PermissionError as e:  
                    continue
process_pdf_files(main_folder, image_folder)


# PDF to images

In [ ]:
from ultralyticsplus import YOLO, render_result
import os
from pdf2image import convert_from_path
model = YOLO('keremberke/yolov8m-table-extraction')
main_folder = r"C:\Users\amana\OneDrive\Desktop\ASIA\ASIA"
image_folder = r"C:\Users\amana\OneDrive\Desktop\pdf images"

def process_pdf_files(folder_path, output_folder):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".pdf"):
                pdf_path = os.path.join(root, file)  # Full path to the PDF file
                pdf_image_folder = os.path.join(output_folder, os.path.relpath(root, folder_path), file)
                os.makedirs(pdf_image_folder, exist_ok=True)
                try:
                    images = convert_from_path(pdf_path, poppler_path=r"C:\Users\amana\Downloads\Release-23.11.0-0\poppler-23.11.0\Library\bin")
                    for index, image in enumerate(images):
                        image_file_path = os.path.join(pdf_image_folder, f"{index}.jpg")
                        image.save(image_file_path)
                            
                except PermissionError as e:    # some of the files are not pdf (html, microsoft edge files that need permission to be accessed)
                    continue

process_pdf_files(main_folder, image_folder)

    

# Saving images with tables with max detection as 1 ( saving the table with the max confidence)

In [ ]:
from ultralyticsplus import YOLO, render_result
import os

model = YOLO('keremberke/yolov8m-table-extraction')
model.overrides['max_det'] = 1     # override the # of table detection hyperparameter

input_folder = r"C:\Users\amana\OneDrive\Desktop\pdf images"
output_folder = r"C:\Users\amana\OneDrive\Desktop\new pdf images with tables"

def process_images(input_folder, output_folder):
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith((".jpg", ".png", ".jpeg")):  
                image_file_path = os.path.join(root, file)  # Full path to the image file
                output_subfolder = os.path.join(output_folder, os.path.relpath(root, input_folder))
                os.makedirs(output_subfolder, exist_ok=True)
                output_file_path = os.path.join(output_subfolder, file)
                try:
                    result = model.predict(image_file_path)
                    render = render_result(model=model, image=image_file_path, result=result[0])
                    if len(result[0].boxes.xyxy) > 0:
                        render.save(output_file_path)
                except FileNotFoundError as e:
                    continue
process_images(input_folder, output_folder)

# finding time ( time for each pdf processing )

In [9]:
                                        ####                                    ####
                                            ####                           ####
                                                ####  THIS CODE WORKS  ####


# import os
# import time
# import pandas as pd
# from ultralyticsplus import YOLO

# def process_images(folder_path, model):
#     processing_times = []
#     for root, dirs, _ in os.walk(folder_path):
#         for dir_name in dirs:
#             if dir_name.endswith(".pdf"):
#                 pdf_folder_path = os.path.join(root, dir_name)
#                 start_time = time.time()
#                 for file in os.listdir(pdf_folder_path):
#                     if file.endswith((".jpg", ".png", ".jpeg")):
#                         image_file_path = os.path.join(pdf_folder_path, file)
#                         try:
#                             model.predict(image_file_path)
#                         except FileNotFoundError as e:
#                             continue
#                 end_time = time.time()
#                 processing_time = end_time - start_time
#                 processing_times.append((dir_name, processing_time))
#     return processing_times

# model = YOLO('keremberke/yolov8m-table-extraction')
# model.overrides['max_det'] = 1

# input_folder = r"C:\Users\amana\OneDrive\Desktop\j"
# processing_times = process_images(input_folder, model)

# df = pd.DataFrame(processing_times, columns=['PDF Folder Name', 'Time Taken'])
# df.to_excel(r"C:\Users\amana\OneDrive\Desktop\Book2.xlsx", index=False)


# detecting table, cropping out table and then using ocr on the cropped image

In [11]:
import paddleocr
import ultralyticsplus
import cv2
import os
from ultralyticsplus import YOLO, render_result
from paddleocr import PaddleOCR, draw_ocr
ocr_model=PaddleOCR(lang='en')
yolo_model = YOLO('keremberke/yolov8m-table-extraction')
yolo_model.overrides['max_det'] = 1

[2024/04/14 22:58:31] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\amana/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\amana/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_len

In [12]:
def crop_region(image, box):
    x_min, y_min, x_max, y_max = [int(item) for item in box]
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(image.shape[1], x_max)
    y_max = min(image.shape[0], y_max)
    cropped_image = image[y_min:y_max, x_min:x_max]
    return cropped_image

In [13]:
path=r"C:\Users\amana\OneDrive\Desktop\j\Abbott_Senai Medicals"
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith((".jpg", ".png", ".jpeg")):
            image_file_path = os.path.join(root, file)
            image = cv2.imread(image_file_path)
            result = yolomodel.predict(image_file_path)
            if len(result[0].boxes.xyxy) > 0:
                boxes = result[0].boxes.xyxy.tolist()[0]
                cropped_image = crop_region(image, boxes)  # Assuming only one bounding box per image
                ocr_result = ocr_model.ocr(cropped_image)
                for x in ocr_result[0]:
                    print(x[1][0])
                print('\n')
                print('\n') 

Ultralytics YOLOv8.0.21  Python-3.9.13 torch-2.0.1+cpu CPU
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs
image 1/1 C:\Users\amana\OneDrive\Desktop\j\Abbott_Senai Medicals\FLEXTRONICS PO X94111238.pdf\0.jpg: 640x512 1 bordered, 1404.2ms
Speed: 5.0ms pre-process, 1404.2ms inference, 14.3ms postprocess per image at shape (1, 3, 640, 640)


[2024/04/14 22:59:39] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/14 22:59:40] ppocr DEBUG: dt_boxes num : 14, elapsed : 0.3186619281768799
[2024/04/14 22:59:44] ppocr DEBUG: rec_res num  : 14, elapsed : 3.907348155975342
Type
Standard Purchase Order
Order
X94111238
Revision
1
Order Date
06-OCT-2020
Created By
RASOUL, MICHELLE
Revision Date
08-OCT-2020
Current Buyer
RASOUL,MICHELLE






image 1/1 C:\Users\amana\OneDrive\Desktop\j\Abbott_Senai Medicals\FLEXTRONICS PO X94111238.pdf\1.jpg: 640x512 1 borderless, 1244.7ms
Speed: 2.0ms pre-process, 1244.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


[2024/04/14 22:59:45] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/14 22:59:46] ppocr DEBUG: dt_boxes num : 61, elapsed : 1.1682038307189941
[2024/04/14 23:00:04] ppocr DEBUG: rec_res num  : 61, elapsed : 17.697516918182373
ine
Part Number/Description
Delivery Date/Time
Quantity
UOM
Unit Price
Tax
Amount
(USD)
(USD)
1
Promised:
20EACH
645.78Y
12,915.60
31-DEC-202000:00:00
Needed:
31-DEC-202000:00:00
9000L100-105-DPCBASSY.LOWER CONTROLLER
1-1
Ship To:
Use the ship-to address at the top of page 1
Deliver To:INTERFACE, NIP (20)
2
Promised:
20EACH
473.16Y
9,463.20
31-DEC-202000:00:00
Needed:
31-DEC-202000:00:00
90001090-111-FPCBASSY,TEMPCONTROL OPTICS
2-1
Ship To:
Use the ship-to address at the top of page 1
Deliver To:INTERFACE, NIP (20)
3
Promised:
20EACH
258.59Y
5,171.80
31-DEC-2020 00:00:00
Needed:
31-DEC-202000:00:00
90001080-107-EPCB ASSY.RV Loader/CRA Control
3-1
Ship To:
Use the ship-to address at the top of p

image 1/1 C:\Users\amana\OneDrive\Desktop\j\Abbott_Senai Medicals\FLEXTRONICS PO X94111238.pdf\2.jpg: 640x512 1 borderless, 1083.4ms
Speed: 2.2ms pre-process, 1083.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


[2024/04/14 23:00:05] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/14 23:00:06] ppocr DEBUG: dt_boxes num : 64, elapsed : 0.7483088970184326
[2024/04/14 23:00:24] ppocr DEBUG: rec_res num  : 64, elapsed : 17.86294412612915
Part Number/Description
Delivery Date/Time
Quantity
UOM
Unit Price
Tax
Amount
(USD)
(USD)
Deliver To:INTERFACE,NIP 20
5
Promised:
20EACH
480.78 Y
9,615.60
31-DEC-202000:00:00
Needed:
31-DEC-2020 00:00:00
90001095-106-DPCB ASSY,UPPER CONTROLLER
-1
Ship To:
Use the ship-to address at the top of page 1
Deliver To:INTERFACE,NIP (20)
6
Promised:
20EACH
535.89Y
10,717.80
31-DEC-202000:00:00
Needed:
31-DEC-202000:00:00
90000945-109-BPCB ASSY,CC PRESSURE MONITOR SOC
-1
Ship To:
Use the ship-to address at the top of page 1
Deliver To:INTERFACE,NIP (20)
Promised:
40 EACH
44.17 Y
1,766.80
31-DEC-202000:00:00
Needed:
31-DEC-202000:00:00
90000080-109-BPCB ASSY,ASP/DISP MONITOR SENSOR
-1
Ship To:
Use the ship

image 1/1 C:\Users\amana\OneDrive\Desktop\j\Abbott_Senai Medicals\PO_124_X94111489_3_US (Highlighted).pdf\0.jpg: 640x512 1 bordered, 1273.6ms
Speed: 2.0ms pre-process, 1273.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


[2024/04/14 23:00:25] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/14 23:00:25] ppocr DEBUG: dt_boxes num : 14, elapsed : 0.2011127471923828
[2024/04/14 23:00:29] ppocr DEBUG: rec_res num  : 14, elapsed : 3.728355884552002
Type
Standard Purchase Order
Order
X94111489
Revision
3
Order Date
30-OCT-2020
Created By
RASOUL, MICHELLE
Revision Date
02-NOV-2020
Current Buyer
RASOUL, MICHELLE






image 1/1 C:\Users\amana\OneDrive\Desktop\j\Abbott_Senai Medicals\PO_124_X94111489_3_US (Highlighted).pdf\1.jpg: 640x512 1 borderless, 1142.2ms
Speed: 1.3ms pre-process, 1142.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


[2024/04/14 23:00:30] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/14 23:00:31] ppocr DEBUG: dt_boxes num : 51, elapsed : 1.1714389324188232
[2024/04/14 23:00:47] ppocr DEBUG: rec_res num  : 51, elapsed : 15.416401863098145
Line
Part Number/Description
Delivery Date/Time
Quantity
UOM
Unit Price
Tax
Amount
(USD)
(USD)
1
Promised:
25EACH
470.5Y
11,762.50
10-NOV-202000:00:00
Needed:
10-NOV-2020 00:00:00
THR1H-AB-90001090-111J-E PCB ASSYTEMP CONTROL OPTICS
1-1
Ship To:
Use the ship-to address at the top of page 1
Deliver To:INTERFACE, NIP (25)
2
Promised:
25 EACH
640.86Y
16,021.50
10-NOV-202000:00:00
Needed:
10-NOV-202000:00:00
THR1H-AB-900011O0-105JC PCBASSY.LOWER CONTROLLER
2-1
Ship To:
Use the ship-to address at the top of page 1
Deliver To:INTERFACE,NIP 25
3
Promised:
25 EACH
258.39
Y
6,459.75
10-NOV-2020 00:00:00
Needed:
10-NOV-2020 00:00:00
THR1H-AB-90001080-107J-C PCB ASSY,RV Loader/CRA Control
3-1
Ship To:
Use

# Camelot ( not needed, but also helps in table detection )

In [1]:
# import camelot
# import pandas as pd
# import tempfile
# # Path to the PDF file containing the table
# pdf_path = r"C:\Users\amana\OneDrive\Desktop\ASIA\ASIA\ABR_011_Suzhou\21675.pdf"

# # Use camelot to extract tables from the PDF
# tables = camelot.read_pdf(pdf_path)

# # Iterate through each table extracted
# for idx, table in enumerate(tables):
#     print(f"Table {idx + 1}:")
    
#     # Get the DataFrame of the current table
#     df = table.df
    
#     # Display the content of the DataFrame
#     with tempfile.NamedTemporaryFile(suffix='.xlsx', delete=False) as temp_file:
#         excel_file_path = temp_file.name
#         df.to_excel(excel_file_path, index=False)
#         print("Temporary Excel file saved at:", excel_file_path)


Table 1:
Temporary Excel file saved at: C:\Users\amana\AppData\Local\Temp\tmppth7cvuk.xlsx


# Structure detection(retaining the shape of the table in the read ocr text)

In [2]:
import paddleocr
import ultralyticsplus
import cv2
import os
import numpy as np
from ultralyticsplus import YOLO, render_result
from paddleocr import PaddleOCR, draw_ocr
ocr_model=PaddleOCR(lang='en')
yolo_model = YOLO('keremberke/yolov8m-table-extraction')
yolo_model.overrides['max_det'] = 1

[2024/04/18 08:35:50] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\amana/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\amana/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_len

In [3]:
# function to crop an image
def crop_region(image, box):
    x_min, y_min, x_max, y_max = [int(item) for item in box]
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(image.shape[1], x_max)
    y_max = min(image.shape[0], y_max)
    cropped_image = image[y_min-10:y_max+10, x_min-10:x_max+10]
    return cropped_image

In [22]:
# image_file_path = r"C:\Users\amana\OneDrive\Desktop\pdf images\ABR_011_Suzhou\21675.pdf\1.jpg"
# image = cv2.imread(image_file_path)
# result = yolo_model.predict(image_file_path)
# if len(result[0].boxes.xyxy) > 0:
#     boxes = result[0].boxes.xyxy.tolist()[0]
#     cropped_image = crop_region(image, boxes)
#     gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY) # convert to gray scale
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
#     eroded_image = cv2.erode(gray_image, kernel, iterations=1)  # erosion
#     ret, thresh = cv2.threshold(eroded_image, 150, 255, cv2.THRESH_BINARY) # setting threshold to create binary image
#     contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # fiding contours
#     white_image = 255 * np.ones_like(image) # create a white sheet
#     # Draw contours 
#     #image_with_contours = cropped_image.copy()
#     cell_coordinates = []
    
#     # Filter contours based on area
#     for contour in contours:
#         area = cv2.contourArea(contour)
#         if area in range(5000,28001):  # Adjust the threshold
#             x, y, w, h = cv2.boundingRect(contour)
#             cell_coordinates.append((x, y, x + w, y + h))
#            #cv2.drawContours(image_with_contours, [contour], -1, (0, 255, 0), 2)
#             cv2.rectangle(white_image, (x, y), (x + w, y + h), (0, 0, 0), -1)
            
#     for idx, cell_coords in enumerate(cell_coordinates, start=1):
#         x1, y1, x2, y2 = cell_coords
#         cell_image = cropped_image[y1:y2, x1:x2]
#         ocr_result = ocr_model.ocr(cell_image)
#         if ocr_result:
#             text = ""
#             for line in ocr_result[0]:
#                 text = '\n'.join([text_block[1][0].encode('utf-8').decode('utf-8') for text_block in ocr_result[0]])  # Concatenate each line of text with a newline character

#             font = cv2.FONT_HERSHEY_SIMPLEX
#             font_scale = 0.5
#             (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness=1)
#             max_text_width = x2 - x1 - 20  
#             max_text_height = y2 - y1 - 30  

#             if text_width > max_text_width or text_height > max_text_height:
#                 lines = []
#                 line = ''
#                 for word in text.split():
#                     if cv2.getTextSize(line + ' ' + word, font, font_scale, thickness=2)[0][0] < max_text_width:
#                         line += ' ' + word if line else word
#                     else:
#                         lines.append(line)
#                         line = word
#                 lines.append(line)
#                 text_y = y1 + 30
#                 for line in lines:
#                     cv2.putText(white_image, line, (x1 + 10, text_y), font, font_scale, (255, 255, 255), 2)
#                     text_y += text_height + 5 
#             else:
#                 text_x = x1 + 10 
#                 text_y = y1 + 30  
#                 cv2.putText(white_image, text, (text_x, text_y), font, font_scale, (255, 255, 255), 2)

#     cv2.imshow('White Image with Text', white_image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    
# for i, (x1, y1, x2, y2) in enumerate(cell_coordinates, start=1):
#     print(f"Cell {i}: (x1={x1}, y1={y1}), (x2={x2}, y2={y2})")

image 1/1 C:\Users\amana\OneDrive\Desktop\pdf images\ABR_011_Suzhou\21675.pdf\1.jpg: 640x480 1 bordered, 989.1ms
Speed: 2.6ms pre-process, 989.1ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


[2024/04/17 21:53:28] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/17 21:53:28] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.024445056915283203
[2024/04/17 21:53:29] ppocr DEBUG: rec_res num  : 1, elapsed : 0.14969086647033691
[2024/04/17 21:53:29] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/17 21:53:29] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.029312610626220703
[2024/04/17 21:53:29] ppocr DEBUG: rec_res num  : 2, elapsed : 0.34018778800964355
[2024/04/17 21:53:29] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/17 21:53:29] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.03135824203491211
[2024/04/17 21:53:29] ppocr DEBUG: rec_res num  : 1, elapsed : 0.18701672554016113
[2024/04/17 21:53:29] ppocr WARNING: Since the angle classifier is not initialized, it will not be used dur

In [27]:
# image_file_path = r"C:\Users\amana\OneDrive\Desktop\pdf images\ABR_011_Suzhou\21675.pdf\1.jpg"
# image = cv2.imread(image_file_path)
# result = yolomodel.predict(image_file_path)
# if len(result[0].boxes.xyxy) > 0:
#     boxes = result[0].boxes.xyxy.tolist()[0]
#     cropped_image = crop_region(image, boxes)
#    # pdb.set_trace()
#     #cv2.imshow('table',cropped_image)
#     #blurred_image = cv2.GaussianBlur(cropped_image, (5, 5), 0)
#     #img_dilation = cv2.dilate(cropped_image, kernel, iterations=1)
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
#     eroded_image = cv2.erode(image, kernel, iterations=1)
    
#    # gray_image = cv2.cvtColor(img_dilation, cv2.COLOR_BGR2GRAY)
#     ret, thresh = cv2.threshold(eroded_image, 150, 255, cv2.THRESH_BINARY)
#     contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     #cv2.imshow('dilated',gray_image)
# #     filtered_contours = []
# #     for contour in contours:
# #         area = cv2.contourArea(contour)
# #         if area > 70:  
# #             filtered_contours.append(contour)
            
# #     image_copy = cropped_image.copy()
# #     for contour in filtered_contours:
# #         x, y, w, h = cv2.boundingRect(contour)
# #         cv2.rectangle(image_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)
#     cv.imshow('contours',)
# #     cv2.imshow('Table with Cell Boundaries', image_copy)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [ ]:
                                       ####                                               ####
                                            #### CODE FOR TABLE STRUCTURE RECOGNTION ####
                                       ####                                               #### 

input_folder=r"C:\Users\amana\OneDrive\Desktop\j"
output_folder=r"C:\Users\amana\OneDrive\Desktop\k"
for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith((".jpg", ".png", ".jpeg")):  
                image_file_path = os.path.join(root, file)  # Full path to the image file
                output_subfolder = os.path.join(output_folder, os.path.relpath(root, input_folder))
                os.makedirs(output_subfolder, exist_ok=True)
                output_file_path = os.path.join(output_subfolder, file)
                try:
                    image = cv2.imread(image_file_path)
                    result = yolo_model.predict(image_file_path)
                    if len(result[0].boxes.xyxy) > 0:
                        boxes = result[0].boxes.xyxy.tolist()[0]
                        cropped_image = crop_region(image, boxes)
                        gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY) # convert to gray scale
                        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
                        eroded_image = cv2.erode(gray_image, kernel, iterations=1)  # erosion
                        ret, thresh = cv2.threshold(eroded_image, 150, 255, cv2.THRESH_BINARY) # setting threshold to create binary image
                        contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # fiding contours
                        white_image = 255 * np.ones_like(image) # create a white sheet
                        # Draw contours 
                        #image_with_contours = cropped_image.copy()
                        cell_coordinates = []

                        # Filter contours based on area
                        for contour in contours:
                            area = cv2.contourArea(contour)
                            if area in range(5000,28001):  # Adjust the threshold
                                x, y, w, h = cv2.boundingRect(contour)
                                cell_coordinates.append((x, y, x + w, y + h))
                               #cv2.drawContours(image_with_contours, [contour], -1, (0, 255, 0), 2)
                                cv2.rectangle(white_image, (x, y), (x + w, y + h), (0, 0, 0), -1)

                        for idx, cell_coords in enumerate(cell_coordinates, start=1):
                            x1, y1, x2, y2 = cell_coords
                            cell_image = cropped_image[y1:y2, x1:x2]
                            ocr_result = ocr_model.ocr(cell_image)
                            if ocr_result:
                                text = ""
                                for line in ocr_result[0]:
                                    text = '\n'.join([text_block[1][0].encode('utf-8').decode('utf-8') for text_block in ocr_result[0]])  # Concatenate each line of text with a newline character

                                font = cv2.FONT_HERSHEY_SIMPLEX
                                font_scale = 0.5
                                (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness=1)
                                max_text_width = x2 - x1 - 20  
                                max_text_height = y2 - y1 - 30  

                                if text_width > max_text_width or text_height > max_text_height:
                                    lines = []
                                    line = ''
                                    for word in text.split():
                                        if cv2.getTextSize(line + ' ' + word, font, font_scale, thickness=2)[0][0] < max_text_width:
                                            line += ' ' + word if line else word
                                        else:
                                            lines.append(line)
                                            line = word
                                    lines.append(line)
                                    text_y = y1 + 30
                                    for line in lines:
                                        cv2.putText(white_image, line, (x1 + 10, text_y), font, font_scale, (255, 255, 255), 2)
                                        text_y += text_height + 5 
                                else:
                                    text_x = x1 + 10 
                                    text_y = y1 + 30  
                                    cv2.putText(white_image, text, (text_x, text_y), font, font_scale, (255, 255, 255), 2)
                        cv2.imwrite(output_file_path, white_image) # save the white sheet

                except FileNotFoundError as e:
                    continue

image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21675.pdf\0.jpg: 640x480 1 bordered, 3062.3ms
Speed: 5.1ms pre-process, 3062.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


[2024/04/18 08:42:55] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:42:55] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.052197933197021484
[2024/04/18 08:42:56] ppocr DEBUG: rec_res num  : 1, elapsed : 0.46630001068115234
[2024/04/18 08:42:56] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:42:56] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.04976344108581543
[2024/04/18 08:42:57] ppocr DEBUG: rec_res num  : 2, elapsed : 1.0300467014312744
[2024/04/18 08:42:57] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:42:57] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.057805776596069336
[2024/04/18 08:42:57] ppocr DEBUG: rec_res num  : 1, elapsed : 0.4744682312011719
[2024/04/18 08:42:57] ppocr WARNING: Since the angle classifier is not initialized, it will not be used durin

image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21675.pdf\1.jpg: 640x480 1 bordered, 2981.0ms
Speed: 4.4ms pre-process, 2981.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


[2024/04/18 08:43:21] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:43:21] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.056732177734375
[2024/04/18 08:43:21] ppocr DEBUG: rec_res num  : 1, elapsed : 0.4725494384765625
[2024/04/18 08:43:21] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:43:21] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.04858827590942383
[2024/04/18 08:43:22] ppocr DEBUG: rec_res num  : 2, elapsed : 1.0501534938812256
[2024/04/18 08:43:22] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:43:22] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.05126595497131348
[2024/04/18 08:43:23] ppocr DEBUG: rec_res num  : 1, elapsed : 0.47504401206970215
[2024/04/18 08:43:23] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during th

image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21675.pdf\10.jpg: 640x480 3027.4ms
Speed: 5.0ms pre-process, 3027.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21675.pdf\11.jpg: 640x480 2986.5ms
Speed: 5.6ms pre-process, 2986.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21675.pdf\12.jpg: 640x480 2965.6ms
Speed: 5.4ms pre-process, 2965.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21675.pdf\13.jpg: 640x480 2954.7ms
Speed: 4.3ms pre-process, 2954.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21675.pdf\14.jpg: 640x480 2939.7ms
Speed: 4.4ms pre-process, 2939.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 C:\Users\amana\OneDr

[2024/04/18 08:44:49] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:44:49] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.05201983451843262
[2024/04/18 08:44:50] ppocr DEBUG: rec_res num  : 1, elapsed : 0.4386019706726074
[2024/04/18 08:44:50] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:44:50] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.05256462097167969
[2024/04/18 08:44:51] ppocr DEBUG: rec_res num  : 2, elapsed : 0.8284032344818115
[2024/04/18 08:44:51] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:44:51] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.05603456497192383
[2024/04/18 08:44:51] ppocr DEBUG: rec_res num  : 1, elapsed : 0.389324426651001
[2024/04/18 08:44:51] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during th

image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21678.pdf\1.jpg: 640x480 3038.7ms
Speed: 4.4ms pre-process, 3038.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21678.pdf\10.jpg: 640x480 3009.8ms
Speed: 4.4ms pre-process, 3009.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21678.pdf\11.jpg: 640x480 2913.8ms
Speed: 5.0ms pre-process, 2913.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21678.pdf\12.jpg: 640x480 2999.7ms
Speed: 5.5ms pre-process, 2999.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21678.pdf\13.jpg: 640x480 2837.3ms
Speed: 4.1ms pre-process, 2837.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 C:\Users\amana\OneDri

[2024/04/18 08:45:55] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:45:55] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.0541536808013916
[2024/04/18 08:45:55] ppocr DEBUG: rec_res num  : 1, elapsed : 0.42332959175109863
[2024/04/18 08:45:55] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:45:55] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.05641317367553711
[2024/04/18 08:45:56] ppocr DEBUG: rec_res num  : 2, elapsed : 0.9417870044708252
[2024/04/18 08:45:56] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/04/18 08:45:56] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.06258940696716309
[2024/04/18 08:45:57] ppocr DEBUG: rec_res num  : 1, elapsed : 0.5152633190155029
[2024/04/18 08:45:57] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during t

image 1/1 C:\Users\amana\OneDrive\Desktop\j\ABR_011_Suzhou\21682.pdf\1.jpg: 640x480 3019.1ms
Speed: 5.5ms pre-process, 3019.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
